In [31]:
from torch import nn
import torch.nn.functional as F
import torch

class DilatedConvolutional(nn.Module):
    def __init__(self):
        super(DilatedConvolutional, self).__init__()
        dilations = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
        channels = [1, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]
        self.main = nn.Sequential()
        for i,d in enumerate(dilations):
            self.main.add_module(str(i), nn.Conv1d(in_channels=channels[i],
                                                   out_channels=channels[i+1], 
                                                   kernel_size=2,
                                                   dilation=d))
            self.main.add_module(str(i)+'_relu', nn.ReLU())
        self.main.add_module('pool', nn.AdaptiveAvgPool1d(1))
        self.fc = nn.Linear(channels[-1], channels[-1])
        
    def forward(self, input):
        output = self.main(input).squeeze(-1)
        print(output.shape)
        output = torch.sigmoid(self.fc(output))
        return output

In [56]:
import dataset
import numpy as np
from importlib import reload
reload(dataset)
dset = dataset.EarthquakeDataset(folder='../data/prepared', 
                                 length=100000, 
                                 transforms='demean:raw', 
                                 split='SAC_20100227_Chile_prem')

In [67]:
siamese = dataset.SiameseDataset(dset)
siamese[0]['data'].shape

(1, 106596) 100000
[(0, 0), (0, 0)]
(1, 100000)
(1, 52560) 100000
[(0, 0), (0, 47440)]
(1, 100000)


(2, 100000)

In [ ]:
data_tensor = torch.from_numpy(dset[10]['data']).unsqueeze(0)
print(data_tensor.shape)
net = Net()
output = net(data_tensor)

In [11]:
from torch.utils.data import DataLoader
dataloader = DataLoader(siamese,
                        batch_size=40,
                        num_workers=0,
                        drop_last=True)

In [33]:
x = next(iter(dataloader))
x['data'].shape

torch.Size([40, 2, 10000])

In [100]:
from networks import utils
utils.show_model(net)

Net(
  (main): Sequential(
    (0): Conv1d(1, 16, kernel_size=(2,), stride=(1,), padding=(1,))
    (0_relu): ReLU()
    (1): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(2,), dilation=(2,))
    (1_relu): ReLU()
    (2): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(4,), dilation=(4,))
    (2_relu): ReLU()
    (3): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(8,), dilation=(8,))
    (3_relu): ReLU()
    (4): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(16,), dilation=(16,))
    (4_relu): ReLU()
    (5): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(32,), dilation=(32,))
    (5_relu): ReLU()
    (6): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(64,), dilation=(64,))
    (6_relu): ReLU()
    (7): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(128,), dilation=(128,))
    (7_relu): ReLU()
    (8): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(256,), dilation=(256,))
    (8_relu): ReLU()
    (9): Conv1d(16, 16, kernel

In [27]:
import torch
x = torch.zeros(1).to('cuda')
x.new(x.shape).fill_(1)

tensor([1.], device='cuda:0')